### Question

**Find the friend from which the sum of distances to other friends is the least.**

In [1]:
import numpy as np

In [2]:
names = ['Bob','David','Mary','Skyler']
locations = [(5,2,10),(2,3,5),(19,3,4),(3,5,1)]

friends = list(map(lambda x, y: {'name': x, 'location': y}, names, locations))
friends

[{'name': 'Bob', 'location': (5, 2, 10)},
 {'name': 'David', 'location': (2, 3, 5)},
 {'name': 'Mary', 'location': (19, 3, 4)},
 {'name': 'Skyler', 'location': (3, 5, 1)}]

**Solution in quadratic time complexity**

In [3]:
def calculate_distance(loc1,loc2):
    (x1,y1,z1)=loc1
    (x2,y2,z2)=loc2
    return ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**(1/2)

def calculate_total_distance(friend,friends):
    return sum(map(lambda x: calculate_distance(friend['location'], x['location']), friends))
    
def find_all_distance(friends):
    return list(map(lambda x: calculate_total_distance(x,friends), friends))

def find_host(friends):
    names = list(map(lambda x: x['name'], friends))
    name_distance = np.vstack((names, find_all_distance(friends))).transpose()
    
    result = name_distance[np.argsort(name_distance[:,1])][0]
    return result[0], result[1]
    
result = find_host(friends)
result

('David', '27.528041843981857')

Consider a case when all the points are on a line. The least sum of distances (absolute/manhattan distance in this case) will be for the point that is **median** of numbers. This can be mathematically proved.

In [4]:
data = [2,5,6,3,7,8,93,35]

def calculate_abs_distance(i,j):
    return abs(i-j)
   
def total_abs_distance(datapoint, data): 
    return sum(map(lambda x: calculate_abs_distance(datapoint, x), data))
    
distances = list(map(lambda x: total_abs_distance(x,data), data))

data_dist = (np.vstack((data,distances)).transpose())

result = data_dist[np.argsort(data_dist[:,1])][0]
result[0]

6

However, since our points are in N-dimensional space, we need to calculate something commonly called as **geometric median.**

If we were to minimize the sum of squares of distances instead of sum of distances, the optimum solution will be the **centroid** (can be theoretically found).

And so, an imperfect solution would be to find the closest point to the centroid since we require our point to be one of the friends' house.

In [5]:
locations = np.array(list(map(lambda x: x['location'], friends)))

(p,q,r) = np.mean(locations[:,0]), np.mean(locations[:,1]), np.mean(locations[:,2])

def get_distance(v1,v2):
    return sum((v1-v2)**2)

names = list(map(lambda x: x['name'], friends))

dist_from_center = np.array(list(map(lambda x: get_distance(x, np.array([p,q,r])), locations)))
dist_from_center = np.vstack((names, dist_from_center)).transpose()

result = dist_from_center[np.argsort(dist_from_center)][0]
result[0][0], result[0][1]

('David', '27.625')

### Now, consider if we had a million points in the space. How do we go about these approaches then?

According to **Law of Large Numbers (LLN)**, if you conduct an experiment a number of times, the average of results should be close to expected value. In our case, we are trying to find the mean of all points, and so we can bootstrapp samples and calculate their mean. 

We do this for a number of times and finally compute their average - this value should be close to the actual mean of all points.  

In [6]:
LIMIT = 100
N = 1000000 #population size

X=np.random.choice(np.arange(1,LIMIT), N, replace=True)
Y=np.random.choice(np.arange(1,LIMIT), N, replace=True)
Z=np.random.choice(np.arange(1,LIMIT), N, replace=True)

DATA=np.array([X,Y,Z]).transpose()
np.mean(DATA,axis=0)

array([49.986128, 50.073501, 50.025397])

In [7]:
n = 1000 #sample size
N_ITERATIONS = 100

results=[]

for _ in range(N_ITERATIONS):
    x=np.random.choice(X, n, replace=True)
    y=np.random.choice(Y, n, replace=True)
    z=np.random.choice(Z, n, replace=True)

    x_mean = np.mean(x)
    y_mean = np.mean(y)
    z_mean = np.mean(z)
    results.append([x_mean,y_mean,z_mean])

centroid = np.mean(np.array(results),axis=0)
centroid

array([49.76627, 50.10714, 50.0219 ])

**We see that the population parameters (mean of data points) and average of results are similar.**

So, if we need to find the host which satisfies the objective among a million friends, we can use bootstrapping to **estimate** the centroid of all the friends' locations and find the friend whose location is closest to the estimated centroid.

In [8]:
distances = list(map(lambda x: calculate_distance(x, centroid), DATA))
idx = distances.index(min(distances))

print('Host location: ', DATA[idx])

Host location:  [50 50 50]


Now, let's try if we can find median in a similar manner - using the theory of law of large numbers.

Earlier, we were interested in finding the centroid of population and so we considered centroids of samples and took average. Now, since we are interested in finding the median of data, **we consider median of samples and take average.** 

In [9]:
medoid = np.median(DATA, axis=0) 
medoid

array([50., 50., 50.])

In [10]:
n = 1000 #sample size
N_ITERATIONS = 100

results=[]

for _ in range(N_ITERATIONS):
    x=np.random.choice(X, n, replace=True)
    y=np.random.choice(Y, n, replace=True)
    z=np.random.choice(Z, n, replace=True)

    x_median = np.median(x)
    y_median = np.median(y)
    z_median = np.median(z)
    results.append([x_median,y_median,z_median])

estimated_medoid = np.mean(np.array(results),axis=0)
estimated_medoid

array([49.925, 50.065, 50.08 ])

So, we see that Law of Large Numbers can be used to solve the problem of finding geometric median.